In [2]:
import os
import pandas as pd
import requests

# Load the CSV file
csv_path = "/kaggle/input/sentiment-analysis/reference.csv"  # Change to your actual file path
df = pd.read_csv(csv_path)

# Create a directory for saving images
save_dir = "/kaggle/working/train_images"
os.makedirs(save_dir, exist_ok=True)

# Ensure required columns exist
if "image_url" not in df.columns or "image_name" not in df.columns:
    raise ValueError("CSV file must contain 'image_url' and 'image_name' columns.")

# List to track successfully downloaded image indices
downloaded_indices = []

# Function to download images sequentially
for _, row in df.iterrows():
    img_url = row["image_url"]
    img_name = row["image_name"]
    img_path = os.path.join(save_dir, img_name)

    try:
        response = requests.get(img_url, stream=True, timeout=10)
        if response.status_code == 200:
            with open(img_path, "wb") as img_file:
                img_file.write(response.content)
            print(f"Downloaded: {img_name}")

            # Extract image number (assuming format "image_number.extension")
            image_number = os.path.splitext(img_name)[0].split("_")[1]
            downloaded_indices.append(image_number)

        else:
            print(f"Failed to download: {img_url} (Status Code: {response.status_code})")

    except Exception as e:
        print(f"Error downloading {img_url}: {e}")

# Save successfully downloaded indices to a CSV file
success_log_path = "downloaded_images.csv"
pd.DataFrame({"downloaded_index": downloaded_indices}).to_csv(success_log_path, index=False)

print(f"All downloads completed! Successfully downloaded indices saved to {success_log_path}.")

Downloaded: image_1.jpg
Downloaded: image_2.jpeg
Downloaded: image_3.JPG
Error downloading https://pics.conservativememes.com/10-year-challenge-sweet-dee-edition-40184302.png: HTTPSConnectionPool(host='pics.conservativememes.com', port=443): Max retries exceeded with url: /10-year-challenge-sweet-dee-edition-40184302.png (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7a942efda3e0>: Failed to resolve 'pics.conservativememes.com' ([Errno -2] Name or service not known)"))
Error downloading https://pics.me.me/10-year-challenge-with-no-filter-47-hilarious-10-year-42949168.png: HTTPSConnectionPool(host='pics.me.me', port=443): Max retries exceeded with url: /10-year-challenge-with-no-filter-47-hilarious-10-year-42949168.png (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7a942efdb370>: Failed to resolve 'pics.me.me' ([Errno -2] Name or service not known)"))
Error downloading https://img.memecdn.com/10-years-challenge-about-huma

KeyboardInterrupt: 

In [3]:
import os
import re

# Specify the directory path
directory_path = '/kaggle/working/train_images'

# Initialize an empty list to store the numbers
numbers = []

# Iterate over the files in the directory
for filename in os.listdir(directory_path):
    # Use regular expression to extract the number from the filename
    match = re.search(r'(\d+)', filename)
    if match:
        # Append the extracted number to the list
        numbers.append(int(match.group(1)))

In [10]:
indx = [n-1 for n in numbers]

In [34]:
import pandas as pd
df = pd.read_csv('/kaggle/input/sentiment-analysis/labels.csv')

In [35]:
df = df.iloc[indx]
df = df.drop('Unnamed: 0', axis = 1)
df.to_csv('/kaggle/working/train.csv', index=False)

In [43]:
from sklearn.preprocessing import LabelEncoder

# Initialize label encoders for categorical variables
sentiment_encoder = LabelEncoder()
humor_encoder = LabelEncoder()
sarcasm_encoder = LabelEncoder()
offensive_encoder = LabelEncoder()
motivational_encoder = LabelEncoder()

# Load and preprocess the data
df = pd.read_csv('/kaggle/working/train.csv')

# Encode categorical variables
df['overall_sentiment'] = sentiment_encoder.fit_transform(df['overall_sentiment'])
df['humour'] = humor_encoder.fit_transform(df['humour'])
df['sarcasm'] = sarcasm_encoder.fit_transform(df['sarcasm'])
df['offensive'] = offensive_encoder.fit_transform(df['offensive'])
df['motivational'] = motivational_encoder.fit_transform(df['motivational'])

In [44]:
df.head()

,image_name,text_ocr,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
0,image_727.jpg,You didn't cry when Bambi's mother died? Yes,You didn't cry when Bambi's mother died? Yes ...,2,0,1,0,1
1,image_1206.jpg,Carry yourself w/the confidence of a girl hold...,Carry yourself w/the confidence of a girl hold...,3,0,3,0,2
2,image_161.jpg,WHAAAAAAAAT IWASNT LISTENING,WHAAAAAAAAT IWASNT LISTENING,3,2,2,1,2
3,image_1237.png,me after taking one feminist theory class Amen...,me after taking one feminist theory class Amen...,3,1,0,0,1
4,image_188.jpg,OH SHIT THINK MY WUDU BROKE,OH SHIT THINK MY WUDU BROKE,2,1,1,1,4


In [67]:
## import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image, UnidentifiedImageError


from PIL import Image

class MemeDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.iloc[idx]['image_name']
        img_path = os.path.join(self.img_dir, img_name)
        
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error loading image {img_name}: {e}")
            # Return a blank image if the file can't be loaded
            image = Image.new('RGB', (224, 224), color='black')

        text = self.df.iloc[idx]['text_corrected']
        
        if self.transform:
            image = self.transform(image)
        
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=MAX_LEN,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )
        
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'image': image,
            'sentiment': torch.tensor(self.df.iloc[idx]['overall_sentiment'], dtype=torch.long),
            'humor': torch.tensor(self.df.iloc[idx]['humour'], dtype=torch.float),
            'sarcasm': torch.tensor(self.df.iloc[idx]['sarcasm'], dtype=torch.float),
            'offensive': torch.tensor(self.df.iloc[idx]['offensive'], dtype=torch.float),
            'motivational': torch.tensor(self.df.iloc[idx]['motivational'], dtype=torch.float)
        }



# Model Architecture
class MemeAnalysisModel(nn.Module):
    def __init__(self):
        super(MemeAnalysisModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Identity()  # Remove the final fully connected layer
        
        self.fusion = nn.Linear(768 + 2048, 512)
        self.dropout = nn.Dropout(0.1)
        
        # Task-specific layers
        self.sentiment_classifier = nn.Linear(512, 5)  
        self.humor_classifier = nn.Linear(512, 4)
        self.sarcasm_classifier = nn.Linear(512, 4)
        self.offensive_classifier = nn.Linear(512, 4)
        self.motivational_classifier = nn.Linear(512, 2)
        
        # Regression heads for scales
        self.humor_scale = nn.Linear(512, 1)
        self.sarcasm_scale = nn.Linear(512, 1)
        self.offensive_scale = nn.Linear(512, 1)
        self.motivational_scale = nn.Linear(512, 1)

    def forward(self, input_ids, attention_mask, image):
        text_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        text_features = text_output.pooler_output
        
        image_features = self.resnet(image)
        
        combined = torch.cat((text_features, image_features), dim=1)
        fused = self.fusion(combined)
        fused = self.dropout(fused)
        
        sentiment = self.sentiment_classifier(fused)
        humor = self.humor_classifier(fused)
        sarcasm = self.sarcasm_classifier(fused)
        offensive = self.offensive_classifier(fused)
        motivational = self.motivational_classifier(fused)
        
        humor_scale = self.humor_scale(fused)
        sarcasm_scale = self.sarcasm_scale(fused)
        offensive_scale = self.offensive_scale(fused)
        motivational_scale = self.motivational_scale(fused)
        
        return sentiment, humor, sarcasm, offensive, motivational, humor_scale, sarcasm_scale, offensive_scale, motivational_scale

In [83]:
# Constants
MAX_LEN = 128
BATCH_SIZE = 32
LEARNING_RATE = 1e-5
NUM_EPOCHS = 3

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data Loading
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = MemeDataset(df, img_dir='/kaggle/working/train_images', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [94]:
import torch
# Model, Loss, and Optimizer
model = MemeAnalysisModel().to(device)
criterion = nn.CrossEntropyLoss()
mse_loss = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

# Training Loop
for epoch in range(NUM_EPOCHS):
    model.train()
    for batch in train_loader:
        ids = batch['ids'].to(device)
        mask = batch['mask'].to(device)
        image = batch['image'].to(device)
        sentiment = batch['sentiment'].to(device)
        humor = batch['humor'].to(device)
        sarcasm = batch['sarcasm'].to(device)
        offensive = batch['offensive'].to(device)
        motivational = batch['motivational'].to(device)
        
        outputs = model(ids, mask, image)
        
        sentiment_loss = criterion(outputs[0], sentiment)
        humor_loss = mse_loss(outputs[1].squeeze(), humor)
        sarcasm_loss = mse_loss(outputs[2].squeeze(), sarcasm)
        offensive_loss = mse_loss(outputs[3].squeeze(), offensive)
        motivational_loss = mse_loss(outputs[4].squeeze(), motivational)
        
        scale_losses = sum(mse_loss(output.squeeze(), target) for output, target in zip(outputs[5:], [humor, sarcasm, offensive, motivational]))
        
        total_loss = sentiment_loss + humor_loss + sarcasm_loss + offensive_loss + motivational_loss + scale_losses
        
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {total_loss.item():.4f}')

Epoch [1/10], Loss: 1.0165
Epoch [2/10], Loss: 0.7177
Epoch [3/10], Loss: 0.5012
Epoch [4/10], Loss: 0.3594
Epoch [5/10], Loss: 0.2645
Epoch [6/10], Loss: 0.1897
Epoch [7/10], Loss: 0.1430
Epoch [8/10], Loss: 0.1076
Epoch [9/10], Loss: 0.0619
Epoch [10/10], Loss: 0.0501


In [96]:
# Assuming you have a validation dataloader called 'validation_loader'
model.eval()  # Set the model to evaluation mode
all_predictions = []
all_labels = []

with torch.no_grad():  # Disable gradient calculation during validation
    for batch in validation_loader:
        ids = batch['ids'].to(device)
        mask = batch['mask'].to(device)
        image = batch['image'].to(device)
        labels = batch['sentiment'].to(device)  # Assuming 'sentiment' is the name of your label

        outputs = model(ids, mask, image)  # Make predictions

        #outputs[0] is the sentiment output in the model from previous code

        all_predictions.append(outputs[0].cpu())  # Move to CPU for processing
        all_labels.append(labels.cpu())


all_predictions = torch.cat(all_predictions, dim=0)
all_labels = torch.cat(all_labels, dim=0)


num_classes = 5  # Replace with the actual number of classes
class_accuracies = calculate_class_accuracy(all_predictions, all_labels, num_classes)

print("Class Accuracies:")
for class_label, accuracy in class_accuracies.items():
    print(f"{class_label}: {accuracy:.4f}")

Class Accuracies:
humour: 0.9654
sarcasm: 0.9233
offensive: 0.9563
motivational: 0.9672
overall_sentiment: 0.9505
